In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sn
import copy
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision

# from sklearn import tree, svm
# from sklearn.pipeline import make_pipeline
# from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split

import seaborn as sn
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = .5
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (16, 10)

In [2]:
# from lutils.fin.autoencoder import Normalizer

In [3]:
# np.set_printoptions(precision=8, suppress=True)

In [4]:
# %matplotlib inline
%matplotlib qt

In [6]:
n_epochs = 40 #100   # number of epochs to run
batch_size = 8  # size of each batch
seq_len = 32

In [7]:
class FutureTrendModule(nn.Module):
    def __init__(self):
        super(FutureTrendModule, self).__init__()
        
        # batch_size 32 1 # batch_size 32 * 16
        self.encoder = nn.Sequential(
            nn.Linear(32, 1024),
#             nn.BatchNorm1d(1024),
            nn.Sigmoid(),
            nn.Linear(1024, 512),
            nn.Sigmoid(),
            nn.Linear(512, 256),
            nn.Sigmoid(),
            nn.Linear(256, 128),
            nn.Sigmoid(),
            nn.Linear(128, 64),
            
#             nn.Softmax(1),
        )
        
#         self.encoder_out = nn.Sequential(
            
#         )
        
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.Sigmoid(),
            nn.Linear(128, 256),
            nn.Sigmoid(),
            nn.Linear(256, 512),
            nn.Sigmoid(),
            nn.Linear(512, 1024),
            nn.Sigmoid(),
#             nn.BatchNorm1d(1024),
            nn.Linear(1024, 32),
        )
        
        
        input_size = 16
        hidden_size = 16
        num_layers = 4
        self.fc_size = hidden_size * seq_len
        
        # input batch_size 300
        self.fc1 = nn.Sequential(
            nn.Linear(2, input_size),
            nn.BatchNorm1d(seq_len),
            nn.ReLU(),
        )
        
#         self.embedding = nn.Embedding(1, 300)
    
        self.lstm = nn.Sequential(
            nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        )
        
        self.dropout = nn.Dropout(0.2)
        
        # input batch_size 64
        self.output_fc = nn.Linear(self.fc_size, 1)
    
    def init_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0)
            elif 'weight_ih' in name:
                nn.init.kaiming_normal_(param)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param)
                    
    def forward(self, x): # 8 32 1
        batch_size = x.shape[0]
        
        encoded = self.encoder(x.reshape(batch_size, -1))
#         print(encoded.shape)
#         encoded = self.encoder_out(encoded.reshape(batch_size, -1))
#         print(encoded.shape)
        decoded = self.decoder(encoded).reshape(batch_size, seq_len, 1)
        
#         print(encoded.shape) # 8 * 64
        
        # mat1 and mat2 shapes cannot be multiplied (256x2 and 1x16)
        fc1 = self.fc1(encoded.reshape(batch_size, seq_len, -1))
        lstm_out, (h_n, c_n) = self.lstm(fc1)
        dropout = self.dropout(lstm_out.reshape(batch_size, -1))
        outs = self.output_fc(dropout)
        
#         print(x)
#         print(encoded)
#         print(decoded)
#         print(fc1)
#         print(lstm_out)
        
        return outs, fc1, lstm_out, h_n, dropout, encoded, decoded

In [8]:
model = FutureTrendModule().to(0)
# model.load_state_dict(torch.load('D:/option/models/model_0612_01_l8_30S_5T_mean_h_n.pt'))

In [9]:
loss_fn = nn.MSELoss().to(0)  # mean square error
loss_autoencoder_fn = nn.MSELoss().to(0)
# loss_fn = nn.SmoothL1Loss().to(0)
# loss_fn = nn.CrossEntropyLoss().to(0)
# optimizer = optim.Adam(model.parameters(), lr=0.000001, betas=(0.9, 0.98), eps=1e-9)
# optimizer = optim.SGD(model.parameters(), lr = 0.000001, momentum=0.09)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1) #

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
lambda1 = lambda epoch: np.power(0.1, epoch) if epoch <= 6 else 1e-9
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

In [10]:
# SAVE_PATH = 'D:/option/datas'
# SAVE_PATH = 'D:/option/l8'

# DATA_PATH = 'l8_1S_1T_mean'
# DATA_PATH = 'l8_10S_1T_mean'
# DATA_PATH = 'l8_30S_1T_mean'
# DATA_PATH = 'l8_30S_5T_mean'
# DATA_PATH = 'l8_60S_10T_mean'
# DATA_PATH = 'l8_30S_7T_mean'
# DATA_PATH = '30S_15T_mean'

# DATA_PATH = '1S_1T'
# DATA_PATH = '5S_1T'
# DATA_PATH = '10S_3T'
DATA_PATH = '30S_15T'

In [11]:
# SAVE_PATH = os.path.join('D:/option/l8_autoencoder/', DATA_PATH)
# SAVE_PATH = os.path.join('D:/option/master/', DATA_PATH)

# SAVE_PATH = os.path.join('D:/option/master/diff', DATA_PATH)

SAVE_PATH = os.path.join('D:/option/master/rate', DATA_PATH)

In [12]:
class TimeSeriesDataset(Dataset):
    def __init__(self, x, y):
#         x = np.log(x)
#         y = np.log(y)
        x = np.expand_dims(x, 2) # in our case, we have only 1 feature, so we need to convert `x` into [batch, sequence, features] for LSTM
        self.x = x.astype(np.float32)
        self.y = y.astype(np.float32)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

In [13]:
writer = SummaryWriter('runs/log')

In [70]:
# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []

step = 0
# training loop
for epoch in range(n_epochs):

#     total_size = 0
    
#     for ix in range(0, len(dfs_x)-20):
    for j, f in enumerate(os.listdir(SAVE_PATH)):
        if not f.startswith('SHFE.rb'):
            continue
        
        store = pd.HDFStore(os.path.join(SAVE_PATH, f), 'r')
#         X = normalize(store['X'].values)
        X = store['X'].values
        y = store['y']['y'].values
        store.close()
        
        split_index = int(y.shape[0] * .8)
        
        data_x_train = X[:split_index]
        data_x_val = X[split_index:]
        data_y_train = y[:split_index]
        data_y_val = y[split_index:]
        
        dataset_train = TimeSeriesDataset(data_x_train, data_y_train)
        dataset_val = TimeSeriesDataset(data_x_val, data_y_val)
        
        train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
        
        model.train()
        total_size = data_y_train.shape[0]
        running_loss = 0.0
        for idx, (X_batch, y_batch) in enumerate(train_dataloader, 1):
            X_batch = X_batch.to(0)
            y_batch = y_batch.to(0)
            
            optimizer.zero_grad()
            
            y_pred, fc1, lstm1, h_n, dropout, encoded, decoded = model(X_batch)
            loss1 = loss_fn(y_pred.reshape(-1), y_batch)
            loss2 = loss_autoencoder_fn(decoded, X_batch)
            
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * X_batch.shape[0]
            
            step += X_batch.shape[0]
            
#             if step % 8000 == 0:
            if idx == 1 and j % 4 == 0:
                writer.add_scalar('loss/loss', loss.item(), step)
                writer.add_scalar('loss/total_loss', running_loss / (total_size), step)

                writer.add_histogram('train/01_x', torchvision.utils.make_grid(X_batch).data.cpu(), step)
                writer.add_histogram('train/02_fc1', torchvision.utils.make_grid(fc1).data.cpu(), step)
                writer.add_histogram('train/03_lstm1', torchvision.utils.make_grid(lstm1).data.cpu(), step)
                writer.add_histogram('train/04_h_n', torchvision.utils.make_grid(h_n).data.cpu(), step)
                writer.add_histogram('train/05_dropout', torchvision.utils.make_grid(dropout).data.cpu(), step)
                
                writer.add_histogram('train/06_encoded', torchvision.utils.make_grid(encoded).data.cpu(), step)
                writer.add_histogram('train/07_decoded', torchvision.utils.make_grid(decoded).data.cpu(), step)

                writer.add_histogram('train/09_pred', torchvision.utils.make_grid(y_pred).data.cpu(), step)

#         print('Epoch: [{}/{}], [{}/{}], Loss: {:.8f}'.format(epoch + 1, n_epochs, start+10, total_size, running_loss / (X_batch.shape[0] * i)))
#         print('Epoch: [{}/{}], {}, Loss: {:.8f}'.format(epoch + 1, n_epochs, ix, running_loss / (total_size)))
#         print('Epoch: [{}/{}], {}, Loss: {:.8f}'.format(epoch + 1, n_epochs, f, running_loss / (total_size)))
        
        val_dataloader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
        
        model.eval()
        eval_loss = 0.0
        for idx, (X_batch, y_batch) in enumerate(val_dataloader, 1):
            X_batch = X_batch.to(0)
            y_batch = y_batch.to(0)
            
            y_pred, fc1, lstm1, h_n, dropout, encoded, decoded = model(X_batch)
            loss = loss_fn(y_pred.reshape(-1), y_batch).cpu().detach().numpy()
            
            eval_loss += loss.item() * y_batch.shape[0]
        eval_loss = eval_loss / data_y_val.shape[0]
        
#         print('eval loss {:.8f}'.format(eval_loss))
        lr = scheduler.get_last_lr()[0]
        print('Epoch: [{}/{}], {}, Loss: {:.8f}, Eval loss {:.8f}, lr:{:.16f}'.format(epoch + 1, n_epochs, f, running_loss / (total_size), eval_loss, lr))
        if eval_loss < best_mse:
            best_mse = eval_loss
            best_weights = copy.deepcopy(model.state_dict())
    
    scheduler.step()
    
# restore model and return best accuracy
model.load_state_dict(best_weights)

Epoch: [1/40], SHFE.rb.2021-04-06.10S.3T.20703.h5, Loss: 0.21851604, Eval loss 0.04804565, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-07.10S.3T.20703.h5, Loss: 0.18929020, Eval loss 0.01309261, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-08.10S.3T.20703.h5, Loss: 0.12030115, Eval loss 0.01740765, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-09.10S.3T.20703.h5, Loss: 0.10009616, Eval loss 0.04762886, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-12.10S.3T.20703.h5, Loss: 0.12120969, Eval loss 0.01059737, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-13.10S.3T.20703.h5, Loss: 0.09055629, Eval loss 0.02078944, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-14.10S.3T.20703.h5, Loss: 0.06793329, Eval loss 0.01097326, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-15.10S.3T.20703.h5, Loss: 0.09324731, Eval loss 0.16175124, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-04-16.10S.3T.20703.h5, Loss: 0.10411578, Eval loss 0.01302469, lr:0.

Epoch: [1/40], SHFE.rb.2021-07-23.10S.3T.20703.h5, Loss: 0.09538115, Eval loss 0.01962161, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-26.10S.3T.20703.h5, Loss: 0.12558141, Eval loss 0.01153417, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-27.10S.3T.20703.h5, Loss: 0.12934905, Eval loss 0.01597463, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-28.10S.3T.20703.h5, Loss: 0.10312112, Eval loss 0.00808496, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-29.10S.3T.20703.h5, Loss: 0.08289332, Eval loss 0.01802685, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-07-30.10S.3T.20703.h5, Loss: 0.17721610, Eval loss 0.12367413, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-08-02.10S.3T.20703.h5, Loss: 0.20987338, Eval loss 0.02489832, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-08-03.10S.3T.20703.h5, Loss: 0.09887844, Eval loss 0.02172367, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-08-04.10S.3T.20703.h5, Loss: 0.09727094, Eval loss 0.04485324, lr:0.

Epoch: [1/40], SHFE.rb.2021-11-12.10S.3T.20703.h5, Loss: 0.11390368, Eval loss 0.03200033, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-15.10S.3T.20703.h5, Loss: 0.13448905, Eval loss 0.01837468, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-16.10S.3T.20703.h5, Loss: 0.08432799, Eval loss 0.01844801, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-17.10S.3T.20703.h5, Loss: 0.07228957, Eval loss 0.01172414, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-18.10S.3T.20703.h5, Loss: 0.06367956, Eval loss 0.02201056, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-19.10S.3T.20703.h5, Loss: 0.12362872, Eval loss 0.03963278, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-22.10S.3T.20703.h5, Loss: 0.07687558, Eval loss 0.01429297, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-23.10S.3T.20703.h5, Loss: 0.10367556, Eval loss 0.01857340, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2021-11-24.10S.3T.20703.h5, Loss: 0.07897317, Eval loss 0.01062656, lr:0.

Epoch: [1/40], SHFE.rb.2022-03-03.10S.3T.20703.h5, Loss: 0.06383794, Eval loss 0.01543501, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-04.10S.3T.20703.h5, Loss: 0.04590706, Eval loss 0.00944368, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-07.10S.3T.20703.h5, Loss: 0.10938064, Eval loss 0.04473106, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-08.10S.3T.20703.h5, Loss: 0.11580027, Eval loss 0.02156877, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-09.10S.3T.20703.h5, Loss: 0.08003526, Eval loss 0.01188688, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-10.10S.3T.20703.h5, Loss: 0.05194582, Eval loss 0.01335143, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-11.10S.3T.20703.h5, Loss: 0.04488579, Eval loss 0.01257336, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-14.10S.3T.20703.h5, Loss: 0.05016989, Eval loss 0.01321381, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-03-15.10S.3T.20703.h5, Loss: 0.07934586, Eval loss 0.01212472, lr:0.

Epoch: [1/40], SHFE.rb.2022-06-22.10S.3T.20703.h5, Loss: 0.05389455, Eval loss 0.01236894, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-23.10S.3T.20703.h5, Loss: 0.04500555, Eval loss 0.01371513, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-24.10S.3T.20703.h5, Loss: 0.04887479, Eval loss 0.04334308, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-27.10S.3T.20703.h5, Loss: 0.02590879, Eval loss 0.00748440, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-28.10S.3T.20703.h5, Loss: 0.02581960, Eval loss 0.01057810, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-29.10S.3T.20703.h5, Loss: 0.03563739, Eval loss 0.00872006, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-06-30.10S.3T.20703.h5, Loss: 0.04864432, Eval loss 0.01072340, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-07-01.10S.3T.20703.h5, Loss: 0.03954117, Eval loss 0.01882493, lr:0.0010000000000000
Epoch: [1/40], SHFE.rb.2022-07-04.10S.3T.20703.h5, Loss: 0.03863047, Eval loss 0.02513161, lr:0.

...

Epoch: [3/40], SHFE.rb.2022-11-04.10S.3T.20703.h5, Loss: 0.01526980, Eval loss 0.00501115, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-11-07.10S.3T.20703.h5, Loss: 0.01414601, Eval loss 0.00428505, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-11-08.10S.3T.20703.h5, Loss: 0.01317579, Eval loss 0.00451582, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-11-09.10S.3T.20703.h5, Loss: 0.01429090, Eval loss 0.00527957, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-11-10.10S.3T.20703.h5, Loss: 0.01640827, Eval loss 0.00436374, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-11-11.10S.3T.20703.h5, Loss: 0.01540837, Eval loss 0.00498275, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-11-14.10S.3T.20703.h5, Loss: 0.01707695, Eval loss 0.00456967, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-11-15.10S.3T.20703.h5, Loss: 0.01295248, Eval loss 0.00430091, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2022-11-16.10S.3T.20703.h5, Loss: 0.01286180, Eval loss 0.00593697, lr:0.

Epoch: [3/40], SHFE.rb.2023-02-23.10S.3T.20703.h5, Loss: 0.01844171, Eval loss 0.00442010, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2023-02-24.10S.3T.20703.h5, Loss: 0.01601937, Eval loss 0.00717085, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2023-02-27.10S.3T.20703.h5, Loss: 0.01708014, Eval loss 0.00432315, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2023-02-28.10S.3T.20703.h5, Loss: 0.01292981, Eval loss 0.00312886, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2023-03-01.10S.3T.20703.h5, Loss: 0.01256896, Eval loss 0.00283522, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2023-03-02.10S.3T.20703.h5, Loss: 0.01121382, Eval loss 0.00343909, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2023-03-03.10S.3T.20703.h5, Loss: 0.00964259, Eval loss 0.00331940, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2023-03-06.10S.3T.20703.h5, Loss: 0.01210680, Eval loss 0.00370163, lr:0.0000100000000000
Epoch: [3/40], SHFE.rb.2023-03-07.10S.3T.20703.h5, Loss: 0.01149782, Eval loss 0.00265647, lr:0.

Epoch: [4/40], SHFE.rb.2021-05-19.10S.3T.20703.h5, Loss: 0.13430989, Eval loss 0.07779912, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-05-20.10S.3T.20703.h5, Loss: 0.13187874, Eval loss 0.04929062, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-05-21.10S.3T.20703.h5, Loss: 0.08398629, Eval loss 0.07078175, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-05-24.10S.3T.20703.h5, Loss: 0.17007750, Eval loss 0.02461569, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-05-25.10S.3T.20703.h5, Loss: 0.06186711, Eval loss 0.02005246, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-05-26.10S.3T.20703.h5, Loss: 0.09577386, Eval loss 0.03232511, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-05-27.10S.3T.20703.h5, Loss: 0.06430187, Eval loss 0.02613414, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-05-28.10S.3T.20703.h5, Loss: 0.09115030, Eval loss 0.03028033, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-05-31.10S.3T.20703.h5, Loss: 0.11635851, Eval loss 0.03066815, lr:0.

Epoch: [4/40], SHFE.rb.2021-08-31.10S.3T.20703.h5, Loss: 0.03870391, Eval loss 0.01206431, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-09-01.10S.3T.20703.h5, Loss: 0.04928610, Eval loss 0.01312965, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-09-02.10S.3T.20703.h5, Loss: 0.06930941, Eval loss 0.01365812, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-09-03.10S.3T.20703.h5, Loss: 0.04884499, Eval loss 0.01222892, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-09-06.10S.3T.20703.h5, Loss: 0.05787949, Eval loss 0.01702906, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-09-07.10S.3T.20703.h5, Loss: 0.04826196, Eval loss 0.01340938, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-09-08.10S.3T.20703.h5, Loss: 0.04261063, Eval loss 0.01580727, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-09-09.10S.3T.20703.h5, Loss: 0.05392186, Eval loss 0.01753594, lr:0.0000010000000000
Epoch: [4/40], SHFE.rb.2021-09-10.10S.3T.20703.h5, Loss: 0.05778876, Eval loss 0.01343752, lr:0.

KeyboardInterrupt: 

In [71]:
model.load_state_dict(best_weights)

<All keys matched successfully>

In [72]:
y_pred

tensor([[ 0.4673],
        [ 0.3943],
        [ 0.1771],
        [-0.0052],
        [ 0.3573],
        [ 0.0145],
        [ 0.1984],
        [ 0.0720]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [73]:
# torch.save(model.state_dict(), 'D:/option/models/model_0613_01_%s.pt' % DATA_PATH)
# torch.save(model.state_dict(), 'D:/option/models/model_0613_all_lstm4.pt')
# torch.save(model.state_dict(), 'D:/option/models/model_0616_all_lstm.pt')

In [74]:
# model.load_state_dict(torch.load('D:/option/models/model_0612_01_l8_30S_5T_mean_lstm4.pt'))

In [75]:
# model.load_state_dict(torch.load('D:/option/models/model_0613_all_lstm4.pt'))
# model.load_state_dict(torch.load('D:/option/models/model_0616_all_lstm.pt'))

In [76]:
# model.eval()
# model.training

In [77]:
# class Autoencoder(nn.Module):
#     def __init__(self): # 1 1
#         super(Autoencoder, self).__init__()
        
#         self.encoder = nn.Sequential(
#             nn.Linear(1, 128),
#             nn.BatchNorm1d(128),
#             nn.Sigmoid(),
#             nn.Linear(128, 64),
#             nn.Sigmoid(),
#             nn.Linear(64, 32),
#             nn.Sigmoid(),
#             nn.Linear(32, 1),
#         )
        
#         self.decoder = nn.Sequential(
#             nn.Linear(1, 32),
#             nn.Sigmoid(),
#             nn.Linear(32, 64),
#             nn.Sigmoid(),
#             nn.Linear(64, 128),
#             nn.Sigmoid(),
#             nn.BatchNorm1d(128),
#             nn.Linear(128, 1),
#         )
        
#     def forward(self, x):
#         encoded = self.encoder(x)
#         return self.decoder(encoded), encoded 

In [78]:
device = 0

In [79]:
# class Normalizer():
#     def __init__(self):
#         self.model = Autoencoder().to(device)
#         self.model.load_state_dict(torch.load('D:/option/models/SHFE.rb_encoder.pt'))

#     def fit_transform(self, x):
#         return self.model.encoder(torch.tensor(x, dtype=torch.float32, device=0).reshape([-1, 1])).cpu().detach().numpy()

#     def inverse_transform(self, x):
#         return self.model.decoder(torch.tensor(x, dtype=torch.float32, device=0).reshape([-1, 1])).cpu().detach().numpy()

# scaler = Normalizer('D:/option/models/SHFE.rb_encoder_256.pt', 0)

In [80]:
# TEST_DIR = os.path.join('D:/option/l8_autoencoder', '%s_test' % DATA_PATH)
TEST_DIR = os.path.join('D:/option/master/diff', '%s_test' % DATA_PATH)

In [84]:
def plot_file(p):
    store = pd.HDFStore(p, 'r')
    df_x = store['X']
    df_y = store['y']
    last_price = store['last_price']
    store.close()
    
    X_test = df_x.values
    y_test = df_y['y'].values
    
    dataset_test = TimeSeriesDataset(X_test, y_test)
    test_dataloader = DataLoader(dataset_test, batch_size=batch_size)
    
    y_preds = []
    for idx, (X_batch, y_batch) in enumerate(test_dataloader, 1):
        y_pred, fc1, lstm1, h_n, dropout, encoded, decoded = model(X_batch.to(0))
        y_preds.append(y_pred.cpu().detach().numpy())
    
    y_pred = np.vstack(y_preds)
#     y_inverse = df_y['y'] * df_y['std'] + df_y['mu'] # (x*self.sd) + self.mu
#     y_inverse = scaler.inverse_transform(df_y['y'].values)
#     print(y_inverse)
#     print(last_price['last_price'].values)

    fig, ax1 = plt.subplots()
    
#     print(y_pred)
#     print(np.log(y_test))
    ax1.plot(y_pred, label='pred', color='b', marker='x')
    ax1.plot(y_test, label='real', color='r', marker='.')

    ax2 = ax1.twinx()
    ax2.plot(last_price['last_price'].values, color='g', label='price', marker='1')
#     ax2.plot(y_inverse, color='y', label='inserse_price', marker='2')


    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)

    fig.tight_layout()
    plt.show()

In [85]:
%matplotlib qt

In [86]:
# plot_file(os.path.join('D:/option/', 'SHFE_rb_mu.h5'))
# plot_file(os.path.join('D:/option/', 'SHFE_rb_mu_day.h5'))

# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-10.41384.h5'))
# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.30S.15T.20704.h5'))
plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.10S.3T.20704.h5'))

In [59]:
# SHFE.rb.2023-05-04.41390.h5
# SHFE.rb.2023-05-10.41384.h5
store = pd.HDFStore(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-10.41384.h5'), 'r')
df_x = store['X']
df_y = store['y']
last_price = store['last_price']
store.close()

In [60]:
# X_test = normalize(df_x.values)
X_test = df_x.values
y_test = df_y['y'].values

In [61]:
dataset_test = TimeSeriesDataset(X_test, y_test)

In [62]:
test_dataloader = DataLoader(dataset_test, batch_size=batch_size)

In [63]:
y_preds = []
for idx, (X_batch, y_batch) in enumerate(test_dataloader, 1):
    y_pred, fc1, lstm1, h_n, dropout = model(X_batch.to(0))
    y_preds.append(y_pred.cpu().detach().numpy())

In [64]:
y_pred = np.vstack(y_preds)

In [65]:
y_inverse = df_y['y'] * df_y['std'] + df_y['mu'] # (x*self.sd) + self.mu

In [66]:
# plt.plot(y_pred, label='pred', marker='x')
# plt.plot(y_test, label='real', marker='.')
# plt.legend()
# plt.show()

In [67]:
%matplotlib qt

In [70]:
fig, ax1 = plt.subplots()

ax1.plot(y_pred, label='pred', color='b', marker='x')
ax1.plot(y_test, label='real', color='r', marker='.')

ax2 = ax1.twinx()
ax2.plot(last_price['last_price'].values, color='g', label='price', marker='1')
ax2.plot(y_inverse.values, color='y', label='inserse_price', marker='2')


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

fig.tight_layout()
plt.show()

In [71]:
%matplotlib inline

In [72]:
TEST_DIR = os.path.join('D:/option/', '%s_test' % DATA_PATH)

In [73]:
for f in os.listdir(TEST_DIR):
    print(f)
    store = pd.HDFStore(os.path.join(TEST_DIR, f), 'r')
    df_x = store['X']
    df_y = store['y']
    last_price = store['last_price']
    store.close()
    
    X_test = df_x.values
    y_test = df_y['y'].values
    y_inverse = df_y['y'] * df_y['std'] + df_y['mu']
    
    dataset_test = TimeSeriesDataset(X_test, y_test)
    test_dataloader = DataLoader(dataset_test, batch_size=batch_size)
    
    y_preds = []
    for idx, (X_batch, y_batch) in enumerate(test_dataloader, 1):
        y_pred, fc1, lstm1, h_n, dropout = model(X_batch.to(0))
        y_preds.append(y_pred.cpu().detach().numpy())
    
    y_pred = np.vstack(y_preds)
    
#     plt.plot(y_pred, label='pred')
#     plt.plot(y_test, label='real')
#     plt.legend()
#     plt.savefig(os.path.join('D:/option/', '%s_test_fig_lstm4' % DATA_PATH, '%s.jpg' % f), dpi=160)
#     plt.close()
    fig, ax1 = plt.subplots()

    ax1.plot(y_pred, label='pred', color='b', marker='x')
    ax1.plot(y_test, label='real', color='r', marker='.')

    ax2 = ax1.twinx()
    ax2.plot(last_price['last_price'].values, color='g', label='price', marker='1')
    ax2.plot(y_inverse.values, color='y', label='inserse_price', marker='2')
    
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)

    fig.tight_layout()
    plt.savefig(os.path.join('D:/option/', '%s_test_fig' % DATA_PATH, '%s.jpg' % f), dpi=160)
    plt.close()
    

SHFE.rb.2023-05-04.41390.h5
SHFE.rb.2023-05-05.41398.h5
SHFE.rb.2023-05-08.41404.h5
SHFE.rb.2023-05-09.41399.h5
SHFE.rb.2023-05-10.41384.h5
SHFE.rb.2023-05-11.41406.h5
SHFE.rb.2023-05-12.41403.h5
SHFE.rb.2023-05-15.41392.h5
SHFE.rb.2023-05-16.41392.h5
SHFE.rb.2023-05-17.41389.h5
SHFE.rb.2023-05-18.41391.h5
SHFE.rb.2023-05-19.41398.h5
SHFE.rb.2023-05-22.41403.h5
SHFE.rb.2023-05-23.41404.h5
SHFE.rb.2023-05-24.41403.h5
SHFE.rb.2023-05-25.41407.h5
SHFE.rb.2023-05-26.41402.h5
SHFE.rb.2023-05-29.41378.h5
SHFE.rb.2023-05-30.41384.h5
SHFE.rb.2023-05-31.41393.h5
SHFE.rb.2023-06-01.41353.h5
SHFE.rb.2023-06-02.41400.h5
SHFE.rb.2023-06-05.41391.h5
SHFE.rb.2023-06-06.41401.h5
SHFE.rb.2023-06-07.41406.h5
SHFE.rb.2023-06-08.41400.h5
SHFE.rb.2023-06-09.41392.h5


### l8_30S_15T_mean 0.00028